<a href="https://colab.research.google.com/github/Z4UC/PID/blob/main/PID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import numpy as np
import time

# Sabit parametreler
g = 9.81  # Yerçekimi ivmesi (m/s^2)
length = 2.0  # Tahtravallinin uzunluğu (m)
pivot = length / 2  # Denge noktası
friction = 0.05  # Sürtünme katsayısı
err_arr = []

def calculate_torque(weight_left, weight_right, length_left, length_right):
    """
    Kuvvet momentlerini hesaplar.
    """
    torque_left = weight_left * g * length_left
    torque_right = weight_right * g * length_right
    return torque_left - torque_right

def weight_pid(angle, weight_right):
    global err_arr
    """
    PID denetleyici fonksiyonu. Açıyı dengelemek için sağ ağırlığı ayarlar.
    """
    # PID parametreleri
    Kp = 0.5  # Proportional gain
    Ki = 0.0  # Integral gain
    Kd = 0.0  # Derivative gain

    # Hata hesaplama
    target_angle = 0  # Hedef açı (denge durumu)
    error = angle - target_angle
    err_arr.append(error)
    x = len(err_arr) - 1
    print("error lenght: ", x)

    # PID hesaplamaları
    # Proportional kontrol
    adjustment = Kp * error

    # Integral kontrol
    integral_error = sum(err_arr)  # Integral bileşeni, hata birikimi
    adjustment += Ki * integral_error  # Integral bileşeni ekle

    # Derivative kontrol
    if x > 0:  # Derivative hesaplaması için yeterli veri olmalı
        derivative_error = err_arr[x] - err_arr[x-1]  # Türev (hata farkı)
        adjustment += Kd * derivative_error  # Derivative bileşeni ekle

    if x > 0:
        if err_arr[x] - err_arr[x-1] < 0 and error < 0:
            weight_right = weight_right + adjustment
        elif err_arr[x] - err_arr[x-1] < 0 and error > 0:
            weight_right = weight_right - adjustment
        elif err_arr[x] - err_arr[x-1] > 0 and error > 0:
            weight_right = weight_right + adjustment
        elif err_arr[x] - err_arr[x-1] > 0 and error < 0:
            weight_right = weight_right - adjustment


    # Sağ ağırlığı ayarlama
   #weight_right = weight_right + adjustment

    # Sağ ağırlığın minimum sınırını belirleme
    if weight_right < 0:
        weight_right = 0

    return weight_right

def simulate_seesaw(weight_left, weight_right_initial, time_step=0.01, duration=10):
    """
    Tahtravalli simülasyonu.
    """
    angle = 0  # Başlangıç açısı (radyan)
    angular_velocity = 0  # Açısal hız (radyan/s)

    weight_right = weight_right_initial
    length_left = pivot  # Sol tarafın uzunluğu sabit
    length_right = pivot  # Sağ tarafın uzunluğu sabit

    times = np.arange(0, duration, time_step)

    for t in times:
        torque = calculate_torque(weight_left, weight_right, length_left, length_right)
        angular_acceleration = torque / (weight_left + weight_right)  # Basit bir kütle momenti modeli

        # Sürtünme etkisi
        angular_acceleration -= friction * angular_velocity

        # Hareket denklemleri
        angular_velocity += angular_acceleration * time_step
        angle += angular_velocity * time_step

        # Açıyı sınırlandır (-45° ile +45° arasında)
        if angle > np.radians(45):
            angle = np.radians(45)
            angular_velocity = 0
        elif angle < np.radians(-45):
            angle = np.radians(-45)
            angular_velocity = 0

        # Açıyı ve sürtünmeden dolayı yavaşlamayı konsola yazdır
        print(f"Zaman: {t:.2f}s | Açı: {np.degrees(angle):.2f}° | Sağ Ağırlık: {weight_right:.2f}kg")

        # Sağ ağırlığını kontrol etme
        if abs(np.degrees(angle)) != 0:
            weight_right = weight_pid(np.degrees(angle), weight_right)

        # Simülasyonu yavaşlatmak için bekleme
        time.sleep(0.1)

# Başlangıç değerleri
weight_left = 30  # Sol ağırlık (kg)
weight_right_initial = 35  # Sağ ağırlık başlangıcı (kg)

simulate_seesaw(weight_left, weight_right_initial)

Zaman: 0.00s | Açı: -0.00° | Sağ Ağırlık: 35.00kg
error lenght:  0
Zaman: 0.01s | Açı: -0.01° | Sağ Ağırlık: 35.00kg
error lenght:  1
Zaman: 0.02s | Açı: -0.03° | Sağ Ağırlık: 34.99kg
error lenght:  2
Zaman: 0.03s | Açı: -0.04° | Sağ Ağırlık: 34.98kg
error lenght:  3
Zaman: 0.04s | Açı: -0.06° | Sağ Ağırlık: 34.96kg
error lenght:  4
Zaman: 0.05s | Açı: -0.09° | Sağ Ağırlık: 34.93kg
error lenght:  5
Zaman: 0.06s | Açı: -0.12° | Sağ Ağırlık: 34.88kg
error lenght:  6
Zaman: 0.07s | Açı: -0.15° | Sağ Ağırlık: 34.82kg
error lenght:  7
Zaman: 0.08s | Açı: -0.19° | Sağ Ağırlık: 34.74kg
error lenght:  8
Zaman: 0.09s | Açı: -0.24° | Sağ Ağırlık: 34.65kg
error lenght:  9
Zaman: 0.10s | Açı: -0.28° | Sağ Ağırlık: 34.53kg
error lenght:  10
Zaman: 0.11s | Açı: -0.33° | Sağ Ağırlık: 34.39kg
error lenght:  11
Zaman: 0.12s | Açı: -0.39° | Sağ Ağırlık: 34.22kg
error lenght:  12
Zaman: 0.13s | Açı: -0.44° | Sağ Ağırlık: 34.03kg
error lenght:  13
Zaman: 0.14s | Açı: -0.50° | Sağ Ağırlık: 33.81kg
error le

KeyboardInterrupt: 